In [ ]:
basedir = '/home/idies/workspace/persistent/movies/'
import urllib
from bs4 import BeautifulSoup as bs
import pandas
import numpy as np
import os
import json
import io
from pprint import pprint
print('imported')

In [ ]:
with io.open(basedir+'scripts_index.json', 'r', encoding='utf-8') as f:
    scripts = json.load(f)
print('script index file loaded')
with io.open(basedir+'scripts_notfound.json', 'r', encoding='utf-8') as f:
    notfound = json.load(f)
print('notfound scripts file loaded')

In [ ]:
cnt = 0

for onescript in scripts:
    print('Script number {0:,.0f}: {1:}'.format(cnt, onescript['title']))
    url = onescript['scriptlink']
    onescript['script_file_type'] = url.split('.')[-1]
    
    if (onescript['script_file_type'] == 'html'):
        try:
            page = urllib.request.urlopen(url).read()
        except ValueError:
            print('Eeyore can\'t even read this page: {0:}'.format(onescript['scriptlink']))
            notfound.append(onescript)
        soup = bs(page, "html.parser")
        try:
            onescript['script_text'] = soup.find('pre').text
        except AttributeError:
            print('AttributeError on script {0:,.0f}: {1:}, Eeyore is sad at {2:}'.format(cnt, onescript['title'], onescript['scriptlink']))
            notfound.append(onescript)
    elif (onescript['script_file_type'] == 'pdf'):
        print('Eeyore says it\'s a PDF: {0:}'.format(onescript['scriptlink']))
        theresponse = urllib.request.urlopen(url)
        with open(basedir+'scripts/imsdb/{0:}.pdf'.format(onescript['title']), 'wb') as f:
            f.write(theresponse.read()) 
            onescript['localfile'] = f.name
    else:
        print('Script {0:,.0f}: {1:} Eeyore is sadly adding to notfound.'.format(cnt, onescript['title']))
        notfound.append(onescript)
    cnt = cnt + 1
print('Done!')

In [ ]:
os.chdir(basedir)
with io.open('scripts_notfound_step2.json', 'w', encoding='utf-8') as f:
    json.dump(notfound, f)
    
print('saved notfound output file')

with io.open('scripts.json', 'w', encoding='utf-8') as f:
    json.dump(scripts, f)

print('saved scripts output file')

In [ ]:
with io.open('scripts_notfound.json', 'r', encoding='utf-8') as f:
    nf = json.load(f)
#pprint(nf[0])
for onething in nf:
    print(onething['title'], onething['scriptpagelink'],'\n')

In [ ]:
with io.open('scripts.json', 'r', encoding='utf-8') as f:
    scripts = json.load(f)
print(len(scripts))